# การรวบรวมและวิเคราะห์ข้อมูลทางด้านเกษตรกรรมและสิ่งแวดล้อมผ่านการประยุกต์ใช้ Python ร่วมกับ Google Earth Engine API ในพื้นที่จังหวัดภาคเหนือตอนล่าง

การรวบรวมและวิเคราะห์ข้อมูลทางด้านเกษตรกรรมและสิ่งแวดล้อมผ่านการประยุกต์ใช้ Python ร่วมกับ Google Earth Engine API ในพื้นที่จังหวัดภาคเหนือตอนล่าง เป็นกระบวนการที่ใช้เทคโนโลยีและเครื่องมือทางด้านดิจิทัลเพื่อดึงข้อมูลด้านดินและทางอุตสาหกรรมจาก Google Earth Engine (GEE) ซึ่งเป็นแพลตฟอร์มที่ให้บริการข้อมูลดิจิทัลผ่านการประมวลผลภาพจากดาวเทียม

# ขั้นตอนการดึงข้อมูลและวิเคราะห์ข้อมูลทางด้านเกษตรกรรมและสิ่งแวดล้อมที่ใช้ Python ร่วมกับ Google Earth Engine API มีลักษณะดังนี้:

ฟังก์ชันที่ใช้ติดตั้งแพ็กเกจ (packages) เหล่านี้มีวัตถุประสงค์เพื่อให้สามารถใช้งาน Google Earth Engine (EE) API ร่วมกับ geemap และ folium ได้ในสภาพแวดล้อม Python ของคุณ:

In [2]:
# ติดตั้งแพ็กเกจ Earth Engine API
!pip install earthengine-api

# ติดตั้งแพ็กเกจ geemap
!pip install geemap

# ติดตั้งแพ็กเกจ folium
!pip install folium

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [3]:
# นำเข้าแพ็กเกจ geemap เพื่อให้สามารถใช้งานเครื่องมือแสดงแผนที่จาก Google Earth Engine ได้
import geemap

# นำเข้าแพ็กเกจ folium เพื่อให้สามารถใช้งานเครื่องมือสร้างแผนที่แบบอินเตอร์แอคทีฟได้
import folium

# นำเข้าฟังก์ชัน Map จาก geemap เพื่อให้สามารถสร้างแผนที่เบื้องต้นได้
from geemap import Map

# นำเข้าฟังก์ชัน datetime เพื่อให้สามารถใช้งานฟังก์ชันที่เกี่ยวข้องกับวันที่และเวลาได้
from datetime import datetime

# นำเข้าแพ็กเกจ os เพื่อให้สามารถใช้งานฟังก์ชันที่เกี่ยวข้องกับระบบปฏิบัติการได้
import os

In [4]:
# นำเข้าแพ็กเกจ ee เพื่อให้สามารถใช้งาน Google Earth Engine API ได้
import ee

# เริ่มกระบวนการตรวจสอบสิทธิ์และเชื่อมต่อกับ Google Earth Engine API
# ขั้นตอนนี้จะเรียกใช้หน้าเว็บเพื่อให้ผู้ใช้ทำการล็อกอินและอนุญาตการเข้าถึงข้อมูลจาก Google Earth Engine
ee.Authenticate()

# หลังจากที่ผู้ใช้ล็อกอินและอนุญาตเรียบร้อยแล้ว
# ให้เรียกใช้ฟังก์ชัน Initialize() เพื่อเชื่อมต่อกับ Google Earth Engine API
ee.Initialize()

Enter verification code: 4/1AfJohXkpvM7f2GI_eNMy6p_XddZftE0fYY3gLdmW73W7P4CqbFThA5ydsGU

Successfully saved authorization token.


# ฟังก์ชันสำหรับเรียกขอบเขตของจังหวัดในภาคเหนือตอนล่าง

In [5]:
# ฟังก์ชันสำหรับดึงขอบเขตของจังหวัดในภาคเหนือ
def get_province_boundary():
    # รายชื่อของจังหวัดในภาคเหนือ
    provinces = [
        'Phitsanulok', 'Phichit', 'Phetchabun', 'Kampaeng Phet',
        'Tak', 'Nakhon Sawan', 'Sukhothai', 'Uthai Thani', 'Uttaradit'
    ]
    
    # สร้าง FeatureCollection ของขอบเขตจังหวัด
    province_boundary = ee.FeatureCollection([
        ee.Feature(ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
                   .filter(ee.Filter.inList('ADM1_NAME', provinces))
                   .geometry())
    ])

    return province_boundary


# กำหนดรูปแบบและพารามิเตอร์ของแผนที่
Map = geemap.Map(center=[17.5, 100.0], zoom=6)

# ดึงขอบเขตของจังหวัดในภาคเหนือ
province_boundary = get_province_boundary()

# เพิ่มขอบเขตของจังหวัดลงในแผนที่
Map.addLayer(province_boundary, {}, 'Province Boundary')

# เพิ่มแผนที่ควบคุมเพื่อให้ผู้ใช้สามารถเปลี่ยนแปลงและเลือกชั้นข้อมูลได้
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[17.5, 100.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

# ฟังก์ชันสำหรับดึงข้อมูลภาพถ่ายดาวเทียม Sentinel-2 

โค้ดนี้เป็นฟังก์ชันที่ดึงข้อมูลภาพถ่ายดาวเทียม Sentinel-2 ที่เก็บใน Earth Engine ในช่วงเวลาและพื้นที่ที่กำหนด และจัดการกับข้อมูลนั้นให้อยู่ในรูปแบบที่ต้องการเพื่อการศึกษาหรือการวิเคราะห์ทางด้านสิ่งแวดล้อมต่าง ๆ

In [6]:
# กำหนดปีเริ่มและปีสิ้นสุด
start_year = 2019
end_year = 2023

# สร้างวัตถุวันที่สองปีสำหรับปีเริ่มและปีสิ้นสุด
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year + 1, 1, 1)

# สร้างรายการปี
years = ee.List.sequence(start_year, end_year)

# สร้างรายการเดือน
months = ee.List.sequence(1, 12)

# สร้าง ImageCollection สำหรับ Sentinel-2
dataset = ee.ImageCollection('COPERNICUS/S2_SR')

# กรองภาพ Sentinel-2 ตามขอบเขตของพื้นที่ที่ต้องการศึกษา
# และตามช่วงวันที่ที่กำหนด
image = dataset.filterBounds(province_boundary) \
               .filterDate(start_date, end_date) \
               .sort('CLOUDY_PIXEL_PERCENTAGE', False) \
               .mosaic() \
               .clip(province_boundary)

# สร้างรูปภาพผสมสีจริง (True Color)
trueColor = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# สร้าง ImageCollection โดยกรองภาพ Sentinel-2 ตามพื้นที่และช่วงเวลาที่กำหนด
# จากนั้นทำการเรียงลำดับภาพตามเปอร์เซ็นต์ของความมืด (Cloudy Pixel Percentage) จากมากไปน้อย
# ในที่นี้ใช้ .mosaic() เพื่อเลือกภาพที่มีความมืดน้อยที่สุดใน ImageCollection
# และใช้ .clip() เพื่อตัดภาพให้เหมาะสมกับขอบเขตของพื้นที่ที่กำลังศึกษา

ฟังก์ชันนี้ช่วยในการเตรียมข้อมูลภาพถ่ายดาวเทียม Sentinel-2 ในรูปแบบที่ใช้งานได้ง่ายและเหมาะสมสำหรับการทำงานต่อไปในการวิเคราะห์หรือการแสดงผลบนแผนที่ทางด้านสิ่งแวดล้อมครับ

# การวิเคราะห์ข้อมูลทางด้านเกษตรกรรม 

โค้ดที่ให้มีการคำนวณดัชนีทางการผสมสี (Vegetation Indices) ต่าง ๆ จากภาพ Sentinel-2 ที่ได้จากรายการภาพที่ผ่านกระบวนการกรองและจัดเรียงลำดับแล้ว ซึ่งเป็นดัชนีที่ใช้ในการวิเคราะห์พื้นที่ต่าง ๆ โดยให้ข้อมูลที่มีประโยชน์ดังนี้ :

In [7]:
# คำนวณ Normalized Difference Vegetation Index (NDVI)
# NDVI ถูกใช้เพื่อประมาณการปริมาณของพืชที่มีในพื้นที่
# สูตร: (NIR - RED) / (NIR + RED)
ndvi_sent2 = image.expression(
"(NIR - RED) / (NIR + RED)",
{
    "NIR":image.select("B8"),
    "RED":image.select("B4")
});

# คำนวณ Green Normalized Difference Vegetation Index (GNDVI)
# GNDVI ใช้เพื่อวัดปริมาณของพืชที่มีในพื้นที่ โดยเน้นไปที่สีเขียวของพืช
# สูตร: (NIR - GREEN) / (NIR + GREEN)
gndvi_sent2 = image.expression(
"(NIR - GREEN) / (NIR + GREEN)",
{
    "NIR":image.select("B8").divide(1000),
    "GREEN":image.select("B3").divide(1000)
});

# คำนวณ Enhanced Vegetation Index (EVI)
# EVI ถูกออกแบบมาเพื่อลดผลกระทบจากตัวสิ่งแวดล้อมต่าง ๆ ที่อาจทำให้ NDVI ไม่แม่นยำเท่านั้น
# สูตร: 2.5 * (NIR - RED) / ((NIR + 6*RED - 7.5*BLUE) + 1)
evi_sent2 = image.expression(
 '2.5 * (NIR - RED) / ((NIR + 6*RED - 7.5*BLUE) + 1)', 
{
        'NIR': image.select('B8').divide(1000),
        'RED': image.select('B4').divide(1000),
        'BLUE': image.select('B2').divide(1000)
});

# คำนวณ Advanced Vegetation Index (AVI)
# AVI ถูกออกแบบมาเพื่อดูและวัดปริมาณของพืชที่มีในพื้นที่
# สูตร: ((NIR * (10000 - RED) * (NIR - RED))**(1/3)) * (NIR - RED < 0 ? 0 : 1)
# คำนวณ Advanced Vegetation Index (AVI)
avi_expression = (
    '(nir * (10000 - red) * (nir - red))**(1/3)' +
    ' * (nir - red < 0 ? 0 : 1)'
)

avi_sent2 = image.expression(avi_expression, {
    'nir': image.select('B8'),  
    'red': image.select('B4')   
}).uint16()

# คำนวณ Soil Adjusted Vegetation Index (SAVI)
# SAVI ถูกออกแบบมาเพื่อปรับข้อมูลที่ได้จาก NDVI เพื่อให้ได้ข้อมูลที่แม่นยำมากขึ้น
# สูตร: (NIR - RED) / ((NIR + RED + 0.428) * (1.428))
savi_sent2 = image.expression(
      "(NIR - RED) / ((NIR + RED + 0.428) * (1.428))", 
      {
      'NIR' : image.select('B8').divide(1000),
     'RED' : image.select('B4').divide(1000)
      });

# คำนวณ Normalized Difference Moisture Index (NDMI)
# NDMI ใช้เพื่อประมาณการความชื้นในพื้นที่ โดยใช้ค่าสเปกตรัลในส่วนของ Near-Infrared (NIR) และ Shortwave Infrared (SWIR)
# สูตร: (NIR - SWIR) / (NIR + SWIR)
nir_band = image.select('B8')
swir_band = image.select('B11')

ndmi_sent2 = nir_band.subtract(swir_band).divide(nir_band.add(swir_band)).rename('NDMI');

# คำนวณ Green Coverage Index (GCI)
# GCI ใช้เพื่อวัดปริมาณของพืชที่มีในพื้นที่ โดยเน้นไปที่สีเขียวของพืช
# สูตร: NIR / GREEN - 1
nir_band = image.select('B8')
green_band = image.select('B3')

gci_sent2 = nir_band.divide(green_band).subtract(1).rename('GCI');

# คำนวณ Structure Insensitive Pigment Index (SIPI)
# SIPI ถูกออกแบบมาเพื่อวัดปริมาณของสารละลายในพืช โดยใช้ค่าสเปกตรัลในส่วนของ Near-Infrared (NIR), Red, และ Blue
# สูตร: (NIR - Blue) / (NIR - Red)
nir_band = image.select('B8')
red_band = image.select('B4')
blue_band = image.select('B2')

sipi_sent2 = nir_band.subtract(blue_band).divide(nir_band.subtract(red_band)).rename('SIPI');

# คำนวณ Bare Soil Index (BSI)
# BSI ใช้เพื่อประมาณการปริมาณของพืชที่ไม่มีในพื้นที่ โดยใช้ค่าสเปกตรัลในส่วนของ Red, Near-Infrared (NIR), และ Shortwave Infrared (SWIR)
# สูตร: (Red + SWIR) / (NIR + SWIR)
red_band = image.select('B4')
blue_band = image.select('B2')
nir_band = image.select('B8')
swir_band = image.select('B11')

bsi_numerator = red_band.add(swir_band).subtract(nir_band.add(blue_band));
bsi_denominator = red_band.add(swir_band).add(nir_band.add(blue_band));
bsi_sent2 = bsi_numerator.divide(bsi_denominator).rename('BSI');

# คำนวณ Normalized difference water index (NDWI)
# NDWI ใช้เพื่อประมาณการน้ำที่มีในพื้นที่ โดยใช้ค่าสเปกตรัลในส่วนของ Near-Infrared (NIR) และ Green
# สูตร: (NIR - Green) / (NIR + Green)
green_band = image.select('B3')
nir_band = image.select('B8')

ndwi_sent2 = nir_band.subtract(green_band).divide(nir_band.add(green_band)).rename('NDWI')

โดยการใช้ดัชนีทางการผสมสีเหล่านี้ สามารถสกัดข้อมูลที่มีความหลากหลายเพื่อให้ได้ข้อมูลที่สมบูรณ์แบบเกี่ยวกับพื้นที่นั้น ๆ ในช่วงเวลาที่กำหนด

In [8]:
# กำหนดสี palette
# สีถูกกำหนดเป็นรหัส Hexadecimal สำหรับแต่ละค่าใน display list
# โดย palette นี้จะถูกนำมาใช้ในการแสดงผลภาพ

display = [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
]

# สีถูกกำหนดในรูปแบบ Hexadecimal ซึ่งประกอบด้วยสี่ตัวอักษรหลังเครื่องหมาย # (ตัวอย่างเช่น '040274')
# แต่ละสีจะถูกนำมาใช้เพื่อแสดงผลข้อมูลด้วยสีที่กำหนดไว้ในตำแหน่งที่แตกต่างกัน
# ในกรณีนี้น่าจะเป็นสีที่ใช้ในการแสดงผลข้อมูลที่ได้จากการคำนวณดัชนีทางพืชและสารละลาย

In [9]:
# สร้างแผนที่โดยกำหนดพิกัดศูนย์กลางและระดับการซูม
Map = geemap.Map(center=[16.828023952686376, 100.41638539390868], zoom=8)

# เพิ่มภาพ True Color Composite เข้าไปในแผนที่
Map.addLayer(image, trueColor, 'True Color Composite')

# เพิ่มข้อมูล NDVI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(ndvi_sent2, {'min': -1, 'max': 1, 'palette': display}, 'NDVI with Sentinel-2')

# เพิ่มข้อมูล GNDVI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(gndvi_sent2, {'min': 0, 'max': 1, 'palette': display}, 'GNDVI with Sentinel-2')

# เพิ่มข้อมูล EVI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(evi_sent2, {'min': 0, 'max': 1, 'palette': display}, 'EVI with Sentinel-2')

# เพิ่มข้อมูล AVI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(avi_sent2, {'min': 0, 'max': 5500, 'palette': display}, 'AVI with Sentinel-2')

# เพิ่มข้อมูล SAVI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(savi_sent2, {'min': 0, 'max': 0.4, 'palette': display}, 'SAVI with Sentinel-2')

# เพิ่มข้อมูล NDMI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(ndmi_sent2, {'min': -1, 'max': 1, 'palette': display}, 'NDMI with Sentinel-2')

# เพิ่มข้อมูล GCI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(gci_sent2, {'min': -1, 'max': 1, 'palette': display}, 'GCI with Sentinel-2')

# เพิ่มข้อมูล SIPI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(sipi_sent2, {'min': -1, 'max': 1, 'palette': display}, 'SIPI with Sentinel-2')

# เพิ่มข้อมูล BSI พร้อมกับใช้สีที่กำหนดใน display palette
Map.addLayer(bsi_sent2, {'min': -1, 'max': 1, 'palette': display}, 'BSI with Sentinel-2')

# เพิ่มข้อมูล NDWI พร้อมกับใช้สีที่กำหนดไว้ในรูปแบบที่กำหนด
Map.addLayer(ndwi_sent2, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDWI with Sentinel-2')

# เพิ่มแผนที่ควบคุมเพื่อให้ผู้ใช้สามารถเปลี่ยนแปลงและเลือกชั้นข้อมูลได้
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[16.828023952686376, 100.41638539390868], controls=(WidgetControl(options=['position', 'transparent…

# ฟังก์ชันสำหรับดึงข้อมูลภาพถ่ายดาวเทียม Landsat-7

โค้ดที่ให้มีการสร้างแผนที่โดยใช้ข้อมูล Landsat Surface Reflectance ที่ได้จากรายการภาพที่ผ่านกระบวนการกรองและจัดเรียงลำดับแล้ว ซึ่งเป็นภาพทางสถิติ (median) ของคอลเล็คชัน Landsat ในช่วงเวลาที่กำหนด นอกจากนี้ยังได้มีการปรับปรุงค่าเทสคาลิ่งในภาพเพื่อให้ข้อมูลที่แม่นยำมากขึ้น โดยใช้ฟังก์ชัน applyScaleFactors:

In [10]:
#  กำหนดปีเริ่มและปีสิ้นสุด
start_year = 2019
end_year = 2023

# สร้างวัตถุวันที่สองปีสำหรับปีเริ่มและปีสิ้นสุด
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year + 1, 1, 1)

# สร้างรายการปี
years = ee.List.sequence(start_year, end_year)

# สร้างรายการเดือน
months = ee.List.sequence(1, 12)

# สร้าง ImageCollection สำหรับ Landsat data
collection = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
  .filterDate(start_date, end_date)
  .filterBounds(province_boundary)
  .sort('CLOUD_COVER'))

# คำนวณภาพทางสถิติ (median) จากคอลเล็คชั่น
c = collection.median()

# นิยามฟังก์ชันเพื่อให้มีการปรับปรุงค่าเทสคาลิ่งในภาพ
def applyScaleFactors(collection):
    opticalBands = collection.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = collection.select('ST_*.*').multiply(0.00341802).add(149.0)
    return collection.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# ปรับปรุงค่าเทสคาลิ่งในภาพที่ได้
image = applyScaleFactors(c)

# กำหนดพารามิเตอร์สำหรับการแสดงผล
visualization = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.0,
    'max': 0.25,
}

In [11]:
# Normalized Difference Moisture Index (NDMI):
nir_band = image.select('SR_B4')
swir_band = image.select('SR_B5')

ndmi = nir_band.subtract(swir_band).divide(nir_band.add(swir_band)).rename('NDMI');

# Bare Soil Index (BSI):
red_band = image.select('SR_B5')
blue_band = image.select('SR_B1')
nir_band = image.select('SR_B4')
swir_band = image.select('SR_B3')

bsi_numerator = red_band.add(swir_band).subtract(nir_band.add(blue_band))
bsi_denominator = red_band.add(swir_band).add(nir_band.add(blue_band))
bsi = bsi_numerator.divide(bsi_denominator).rename('BSI');

# Normalized Difference Water Index (NDWI):
SWIR_band = image.select('SR_B5')
nir_band = image.select('SR_B4')

ndwi = nir_band.subtract(SWIR_band).divide(nir_band.add(SWIR_band)).rename('NDWI')

In [12]:
# สร้างแผนที่โดยกำหนดพิกัดศูนย์กลางและระดับการซูม
Map = geemap.Map(center=[16.828023952686376, 100.41638539390868], zoom=8)

# เพิ่มชั้นข้อมูล True Color จาก Landsat-7 ที่ตัดต่อกับขอบเขตของจังหวัด
Map.addLayer(image.clip(province_boundary), visualization, 'True Color with Landsat-7')

# เพิ่มชั้นข้อมูล NDMI (Normalized Difference Moisture Index) ที่ได้คำนวณลงในแผนที่
Map.addLayer(ndmi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': display}, 'NDMI with Landsat-7')

# เพิ่มชั้นข้อมูล BSI (Bare Soil Index) ที่ได้คำนวณลงในแผนที่
Map.addLayer(bsi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': display}, 'BSI with Landsat-7')

# เพิ่มชั้นข้อมูล NDWI (Normalized Difference Water Index) ที่ได้คำนวณลงในแผนที่
Map.addLayer(ndwi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDWI with Landsat-7')

# เพิ่มป้ายกำกับชั้นข้อมูล
Map.addLayerControl()

# แสดงแผนที่
Map


Map(center=[16.828023952686376, 100.41638539390868], controls=(WidgetControl(options=['position', 'transparent…

# ฟังก์ชันสำหรับดึงข้อมูลภาพถ่ายดาวเทียม Landsat-8

โค้ดนี้ใช้ Earth Engine Python API (ผ่าน geemap) เพื่อดึงข้อมูล Landsat Surface Reflectance จาก Google Earth Engine และแสดงผลลัพธ์บนแผนที่โดยใช้ geemap

In [13]:
# กำหนดปีเริ่มต้นและปีสิ้นสุดสำหรับการกรองข้อมูล
start_year = 2019
end_year = 2023

# สร้างวัตถุวันที่สำหรับปีเริ่มต้นและปีสิ้นสุด
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year + 1, 1, 1)

# สร้างรายการปีและรายการเดือนสำหรับการใช้งานในภายหลัง
years = ee.List.sequence(start_year, end_year)
months = ee.List.sequence(1, 12)

# สร้าง ImageCollection สำหรับข้อมูล Landsat
collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
  .filterDate(start_date, end_date)         # กรองข้อมูลในช่วงวันที่ระบุ
  .filterBounds(province_boundary)        # กรองข้อมูลภายในขอบเขตที่ระบุ
  .sort('CLOUD_COVER'))                      # เรียงลำดับคอลเล็คชันตามความคมชัดของเมฆ

# คำนวณภาพที่มีค่ามัธยฐานจากคอลเล็คชัน
c = collection.median()

# นิยามฟังก์ชันเพื่อให้มีการปรับปรุงค่าตัวดัชนีในภาพ
def applyScaleFactors(collection):
    # ปรับขนาดตัวดัชนีทางทฤษฎีสีของภาพด้วยค่าที่ระบุและเพิ่มเข้าไปในคอลเล็คชัน
    opticalBands = collection.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = collection.select('ST_*.*').multiply(0.00341802).add(149.0)
    return collection.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# ปรับขนาดตัวดัชนีในภาพที่ได้
dataset = applyScaleFactors(c)

# กำหนดพารามิเตอร์การแสดงผลสำหรับแผนที่
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.25,
}

โค้ดนี้ทำการคำนวณดัชนีที่สำคัญทางด้านการเกษตรและสิ่งแวดล้อมจากข้อมูล Landsat Surface Reflectance และแสดงผลลัพธ์บนแผนที่โดยใช้ geemap

In [14]:
# คำนวณ Normalized Difference Moisture Index (NDMI)
nir_band = dataset.select('SR_B5')
swir_band = dataset.select('SR_B6')

ndmi = nir_band.subtract(swir_band).divide(nir_band.add(swir_band)).rename('NDMI')

# คำนวณ Bare Soil Index (BSI)
red_band = dataset.select('SR_B6')
blue_band = dataset.select('SR_B2')
nir_band = dataset.select('SR_B5')
swir_band = dataset.select('SR_B4')

bsi_numerator = red_band.add(swir_band).subtract(nir_band.add(blue_band))
bsi_denominator = red_band.add(swir_band).add(nir_band.add(blue_band))
bsi = bsi_numerator.divide(bsi_denominator).rename('BSI')

# คำนวณ Normalized difference water index (NDWI)
SWIR_band = dataset.select('SR_B6')
nir_band = dataset.select('SR_B5')

ndwi = nir_band.subtract(SWIR_band).divide(nir_band.add(SWIR_band)).rename('NDWI')

In [15]:
# แสดงผลลัพธ์บนแผนที่
Map = geemap.Map(center=[16.828023952686376, 100.41638539390868], zoom=8)

# แสดง True Color ของ Landsat-8
Map.addLayer(dataset.clip(province_boundary), visualization, 'True Color with Landsat-8')

# แสดง Normalized Difference Moisture Index (NDMI) บนแผนที่
Map.addLayer(ndmi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': display}, 'NDMI with Landsat-8')

# แสดง Bare Soil Index (BSI) บนแผนที่
Map.addLayer(bsi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': display}, 'BSI with Landsat-8')

# แสดง Normalized Difference Water Index (NDWI) บนแผนที่
Map.addLayer(ndwi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': ['green', 'white', 'blue']}, 'NDWI with Landsat-8')

# เพิ่มเครื่องมือควบคุมเลเยอร์ในแผนที่
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[16.828023952686376, 100.41638539390868], controls=(WidgetControl(options=['position', 'transparent…

# ฟังก์ชันสำหรับดึงข้อมูลภาพถ่ายดาวเทียม Landsat-9

In [16]:
# กำหนดปีเริ่มต้นและปีสิ้นสุด
start_year = 2019
end_year = 2023

# สร้างวัตถุวันที่สองปีสำหรับปีเริ่มต้นและปีสิ้นสุด
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year + 1, 1, 1)

# สร้างรายการปี
years = ee.List.sequence(start_year, end_year)

# สร้างรายการเดือน
months = ee.List.sequence(1, 12)

# สร้าง ImageCollection สำหรับข้อมูล Landsat
collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(province_boundary)
    .sort('CLOUD_COVER'))

# คำนวณภาพมีเดียนจากคอลเล็คชัน
c = collection.median()

# นิยามฟังก์ชันเพื่อปรับปรุงค่าสเกลในภาพ
def applyScaleFactors(collection):
    opticalBands = collection.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = collection.select('ST_*.*').multiply(0.00341802).add(149.0)
    return collection.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# ปรับปรุงค่าสเกลในภาพที่ได้
dataset = applyScaleFactors(c)

# กำหนดพารามิเตอร์สำหรับการแสดงผล
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.25,
}

In [17]:
# คำนวณ Normalized Difference Moisture Index (NDMI)
nir_band = dataset.select('SR_B5')
swir_band = dataset.select('SR_B6')

# ใช้สูตร NDMI: (NIR - SWIR) / (NIR + SWIR)
ndmi = nir_band.subtract(swir_band).divide(nir_band.add(swir_band)).rename('NDMI')

# คำนวณ Bare Soil Index (BSI)
red_band = dataset.select('SR_B6')
blue_band = dataset.select('SR_B2')
nir_band = dataset.select('SR_B5')
swir_band = dataset.select('SR_B4')

# ใช้สูตร BSI: (RED + SWIR - NIR - BLUE) / (RED + SWIR + NIR + BLUE)
bsi_numerator = red_band.add(swir_band).subtract(nir_band.add(blue_band))
bsi_denominator = red_band.add(swir_band).add(nir_band.add(blue_band))
bsi = bsi_numerator.divide(bsi_denominator).rename('BSI')

# คำนวณ Normalized Difference Water Index (NDWI)
SWIR_band = dataset.select('SR_B6')
nir_band = dataset.select('SR_B5')

# ใช้สูตร NDWI: (NIR - SWIR) / (NIR + SWIR)
ndwi = nir_band.subtract(SWIR_band).divide(nir_band.add(SWIR_band)).rename('NDWI')


In [18]:
# ตั้งค่าแผนที่ด้วย geemap
Map = geemap.Map(center=[16.828023952686376, 100.41638539390868], zoom=8)

# เพิ่มชั้นข้อมูลภาพถ่าย Landsat-9 ที่ตัดเขตของจังหวัด
Map.addLayer(dataset.clip(province_boundary), visualization, 'True Color with Landsat-9')

# คำนวณและเพิ่มชั้นข้อมูลดัชนี NDMI โดยให้มีการตัดเขตจังหวัด
Map.addLayer(ndmi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': display}, 'NDMI with Landsat-9')

# คำนวณและเพิ่มชั้นข้อมูลดัชนี BSI โดยให้มีการตัดเขตจังหวัด
Map.addLayer(bsi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': display}, 'BSI with Landsat-9')

# คำนวณและเพิ่มชั้นข้อมูลดัชนี NDWI โดยให้มีการตัดเขตจังหวัด
Map.addLayer(ndwi.clip(province_boundary), {'min': -1, 'max': 1, 'palette': display}, 'NDWI with Landsat-9')

# เพิ่มตัวควบคุมเลเยอร์ในแผนที่
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[16.828023952686376, 100.41638539390868], controls=(WidgetControl(options=['position', 'transparent…

# ฟังก์ชันการคำนวณ Moisture Stress Index (MSI) ด้วยข้อมูล MODIS Terra

คำนวณแผนที่ภาพระบบเรื่อง (Moisture Stress Index - MSI) ด้วยข้อมูล MODIS Terra

In [19]:
# กำหนดปีเริ่มและปีสิ้นสุด
start_year = 2019
end_year = 2023

# สร้างวัตถุวันที่สองปีสำหรับปีเริ่มและปีสิ้นสุด
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year + 1, 1, 1)

# สร้างรายการปี
years = ee.List.sequence(start_year, end_year)

# สร้างรายการเดือน
months = ee.List.sequence(1, 12)

# นำเข้าและกรองชุดข้อมูลพลังสะReflectance ของ MODIS Terra
mod09 = ee.ImageCollection('MODIS/006/MOD09A1')
mod09 = mod09.filterDate(start_date, end_date)

# กำหนดฟังก์ชันเพื่อหน่วงเมฆ
def mask_clouds(image):
    quality = image.select('StateQA')
    mask = image.updateMask(
        quality.bitwiseAnd(1).eq(0)  # ไม่มีเมฆ
        .And(quality.bitwiseAnd(2).eq(0))  # ไม่มีเมฆเงา
    )
    return image.updateMask(mask)

mod09 = mod09.map(mask_clouds)

# ฟังก์ชันสำหรับคำนวณ MSI (Moisture Stress Index)
def calculate_msi(image):
    nir_band = image.select('sur_refl_b02')
    swir_band = image.select('sur_refl_b06')

    msi = swir_band.divide(nir_band).rename('MSI')\
        .set('system:time_start', image.get('system:time_start'))
    return msi

# คำนวณ MSI สำหรับทุกภาพในชุดข้อมูล
MSI = mod09.map(calculate_msi)

# ฟังก์ชันสำหรับคำนวณ MSI รายเดือน
def calculate_monthly_msi(y):
    def calculate_month(m):
        # คำนวณ MSI รายเดือน
        msi = MSI.filter(ee.Filter.calendarRange(y, y, 'year'))\
            .filter(ee.Filter.calendarRange(m, m, 'month'))\
            .mean()

        return msi

    return months.map(calculate_month)

# สร้าง ImageCollection รวม MSI รายเดือน
ic = ee.ImageCollection.fromImages(
    years.map(calculate_monthly_msi).flatten()
)

# สร้างภาพประกอบจาก MSI รายเดือน
alerts = ic.select('MSI').mosaic().clip(province_boundary)

# กำหนดรูปแบบการแสดงผลสำหรับ MSI
msi_vis = {
    'min': 0.25,
    'max': 1,
    'palette':  ['darkblue', 'blue', 'yellow', 'orange', 'red']
}

# สร้างแผนที่
Map = geemap.Map(center=[16.828023952686376, 100.41638539390868], zoom=8)

# เพิ่มชั้นข้อมูล MSI ลงในแผนที่
Map.addLayer(alerts, msi_vis, 'MSI with MODIS')

# เพิ่มตัวควบคุมเลเยอร์ในแผนที่
Map.addLayerControl()

# แสดงแผนที่
Map


Map(center=[16.828023952686376, 100.41638539390868], controls=(WidgetControl(options=['position', 'transparent…

ฟังก์ชันทั้งหมดมีบทบาทในการทำให้โค้ดนี้ทำงานได้อย่างถูกต้องและมีความสมบูรณ์ การแบ่งงานเป็นฟังก์ชันช่วยให้โค้ดอ่านและแก้ไขได้ง่ายขึ้นและทำให้โค้ดมีโครงสร้างที่ดี

# การวิเคราะห์ข้อมูลทางด้านสิ่งแวดล้อม

ได้แก่ ข้อมูลอุณหภูมิ, ข้อมูลปริมาณน้ำฝน และข้อมูลคุณภาพอากาศ เช่น  ฝุ่น PM2.5 และก๊าซที่มีผลกระทบต่อคุณภาพอากาศ เช่น คาร์บอนมอนอกไซด์ (CO), ไนโตรเจนไดออกไซด์ (NO2), และซัลเฟอร์ไดออกไซด์ (SO2) 

# ข้อมูลอุณหภูมิ

โค้ดนี้ใช้ Python ร่วมกับ Google Earth Engine API และ geemap library เพื่อนำเข้าข้อมูลสะสมและวิเคราะห์ข้อมูลสภาพอากาศที่เกี่ยวข้องกับอุณหภูมิผิวดิน (Land Surface Temperature, LST) ในระหว่างปี 2019-2023 ในพื้นที่ที่ถูกกำหนดโดย province_boundary ในภาคเหนือตอนล่างของประเทศไทย

In [20]:
# กำหนดช่วงปีและวันที่: โค้ดเริ่มต้นโดยกำหนดปีเริ่มและปีสิ้นสุด (start_year และ end_year) 
# สร้างวัตถุวันที่สำหรับวันแรกของปีและวันแรกของปีถัดไป (start_date และ end_date) เพื่อให้ง่ายต่อการกรองข้อมูล

# กำหนดปีเริ่มและปีสิ้นสุด
start_year = 2019
end_year = 2023

# สร้างวัตถุวันที่สองปีสำหรับปีเริ่มและปีสิ้นสุด
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year + 1, 1, 1)

# สร้างรายการปี
years = ee.List.sequence(start_year, end_year)

# สร้างรายการเดือน
months = ee.List.sequence(1, 12)

# นำเข้าและกรองข้อมูล MODIS Terra: ใช้ Earth Engine API เพื่อนำเข้าข้อมูล MODIS Terra surface reflectance 
# สำหรับกลางวัน (LST_Day_1km) และกลางคืน (LST_Night_1km) ในช่วงเวลาที่กำหนด
datasetD = ee.ImageCollection('MODIS/006/MYD11A1').filter(ee.Filter.date(start_date, end_date)).select('LST_Day_1km')
datasetN = ee.ImageCollection('MODIS/006/MYD11A1').filter(ee.Filter.date(start_date, end_date)).select('LST_Night_1km')

# คำนวณค่าเฉลี่ยและคลิปข้อมูล: นำเข้าข้อมูลเข้ามาและคำนวณค่าเฉลี่ยของข้อมูล LST ในแต่ละวันของชุดข้อมูล 
# จากนั้นคลิปข้อมูล LST ที่ได้จากคำนวณเฉลี่ยนี้โดยใช้ขอบเขตที่กำหนดโดย province_boundary.
clip_D = datasetD.mean().clip(province_boundary)
clip_N = datasetN.mean().clip(province_boundary)

# กำหนดสีและแสดงผลบนแผนที่: กำหนดสีแบนด์ที่ให้แสดงผลบนแผนที่
band = {
  'min': 13000.0,
  'max': 16500.0,
  'palette': [
    '#00007b', '#0e50f6', '#05fce0', '#1cfe07', '#f2fc03', '#feaf11', '#f46f28', '#fe4709']
}

# สร้างแผนที่ด้วย Geemap (ศูนย์กลางที่ [16.828023952686376, 100.41638539390868], ระดับซูมเป็น 8)
Map = geemap.Map(center=[16.828023952686376, 100.41638539390868], zoom=8)

# เพิ่มชั้นข้อมูล LST (Day) ลงในแผนที่
Map.addLayer(clip_D, band, 'Land Surface Temperature (Day)');

# เพิ่มชั้นข้อมูล LST (Night) ลงในแผนที่
Map.addLayer(clip_N, band, 'Land Surface Temperature (Night)');

# เพิ่มตัวควบคุมเลเยอร์ในแผนที่
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[16.828023952686376, 100.41638539390868], controls=(WidgetControl(options=['position', 'transparent…

ในโค้ดนี้ เราได้นำเข้าข้อมูลอุณหภูมิอากาศ ERA5 Land และทำการคำนวณค่าเฉลี่ยของอุณหภูมิที่ 2 เมตร (temperature_2m) ในช่วงเวลาที่กำหนด จากนั้นทำการลบค่าคงที่ 273.15 เพื่อแปลงหน่วยอุณหภูมิจากเคลวินเซลเซียส (K) เป็นเซลเซียส (Celsius) หลังจากนั้นใช้ focal_mean เพื่อทำการ smoothing ข้อมูล ด้วยการนำข้อมูลในพิกเซลรอบ ๆ ตัวพิกเซลปัจจุบันมาคำนวณค่าเฉลี่ย เพื่อให้ได้ค่าใหม่สำหรับพิกเซลนั้น ๆ ทำให้เกิดการ "smoothing" หรือ "blurring" ขข้อมูลภายในรูปภาพ และท้ายที่สุดทำการเพิ่มแผนที่ Air temperature ที่ได้ลงในแผนที่

In [21]:
# ข้อมูลอุณหภูมิอากาศ Air temperature (อุณหภูมิเฉลี่ย)
dataset = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").filter(ee.Filter.date(start_date, end_date)).select('temperature_2m');
mean = dataset.median().subtract(273.15);

# กำหนดการแสดงผลในแผนที่
vis = {
  'bands': ['temperature_2m'],
  'min': 10.0,
  'max': 35.0,
  'palette': [
    "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
    "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
    "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
  ]
};

# การใช้ focal_mean เป็นการนำข้อมูลในพิกเซลรอบ ๆ ตัวพิกเซลปัจจุบันมาคำนวณค่าเฉลี่ย เพื่อให้ได้ค่าใหม่สำหรับพิกเซลนั้น ๆ 
# ทำให้เกิดการ "smoothing" หรือ "blurring" ข้อมูลภายในรูปภาพ โดยที่ค่าเฉลี่ยนี้จะถูกนำมาใช้แทนค่าในตำแหน่งที่ตัวพิกเซลปัจจุบันอยู่ในแต่ละครั้ง
# กำหนด kernel size
kernel_size = 5

# ใช้ focal_mean เพื่อ smoothing
smooth_mean = mean.focal_mean(radius=kernel_size, kernelType='circle', units='pixels')

# เพิ่มแผนที่ Air temperature ลงในแผนที่
Map.addLayer(smooth_mean.clip(province_boundary), vis, 'Air temperature ERA5 Land Temperature');


# ข้อมูลปริมาณน้ำฝน

ในโค้ดนี้ เรานำเข้าชุดข้อมูล CHIRPS ซึ่งเป็นข้อมูลการปริมาณน้ำฝนราย 5 วัน จาก Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS) และกรองข้อมูลในช่วงเวลาที่กำหนดด้วย filterDate จากนั้นเรานิยามฟังก์ชัน calculate_monthly_precip เพื่อจะคำนวณปริมาณน้ำฝนรายเดือน และสร้าง ImageCollection ที่มีข้อมูลน้ำฝนรายเดือน. ในส่วนของการแสดงผล (precip_vis) เรากำหนดค่าสีและขีดเขียวแดงของแผนที่ที่ต้องการให้สามารถดูปริมาณน้ำฝนได้อย่างชัดเจน

In [29]:
# กำหนดปีเริ่มและปีสิ้นสุด
start_year = 2019
end_year = 2023

# สร้างวัตถุวันที่สองปีสำหรับปีเริ่มและปีสิ้นสุด
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year + 1, 1, 1)

# สร้างรายการปี
years = ee.List.sequence(start_year, end_year)

# สร้างรายการเดือน
months = ee.List.sequence(1, 12)

# นำเข้าชุดข้อมูล CHIRPS (Climate Hazards Group InfraRed Precipitation with Station data)
CHIRPS = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

# กรองชุดข้อมูลสำหรับช่วงเวลาที่เกี่ยวข้อง
CHIRPS = CHIRPS.filterDate(start_date, end_date)

# นิยามฟังก์ชันสำหรับคำนวณปริมาณน้ำฝนรายเดือน
def calculate_monthly_precip(y):
    def calculate_month(m):
        # คำนวณปริมาณน้ำฝนรายเดือน
        precip = CHIRPS.filter(ee.Filter.calendarRange(y, y, 'year')) \
            .filter(ee.Filter.calendarRange(m, m, 'month')) \
            .sum()

        return precip.set('year', y) \
            .set('month', m) \
            .set('system:time_start', ee.Date.fromYMD(y, m, 1))

    return months.map(calculate_month)

# สร้าง ImageCollection ที่มีข้อมูลปริมาณน้ำฝนรายเดือน
monthly_precip = ee.ImageCollection.fromImages(
    years.map(calculate_monthly_precip).flatten()
)

# กำหนดพารามิเตอร์สำหรับการแสดงผล
precip_vis = {
    'min': 0,
    'max': 250,
    'palette': ['white', 'blue', 'darkblue', 'red', 'purple']
}

# สร้างแผนที่ด้วย Geemap (ศูนย์กลางที่ [16.828023952686376, 100.41638539390868], ระดับซูมเป็น 8)
Map = geemap.Map(center=[16.828023952686376, 100.41638539390868], zoom=8)

# เพิ่มชั้นข้อมูล Precipitation ลงในแผนที่
Map.addLayer(monthly_precip.mean().clip(province_boundary),precip_vis,'Precipitation');

# เพิ่มตัวควบคุมเลเยอร์ในแผนที่
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[16.828023952686376, 100.41638539390868], controls=(WidgetControl(options=['position', 'transparent…

In [30]:
# กำหนดวันที่เริ่มต้นและสิ้นสุด
start_date = ee.Date('2019-01-01')
end_date = ee.Date('2023-12-31')

# นำเข้าข้อมูล GlobalSurfaceWater
gsw = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')

# กรองข้อมูลในช่วงเวลาที่ต้องการ
gsw_filtered = gsw.select('occurrence').updateMask(gsw.select('occurrence')).clip(province_boundary)

# เพิ่มแผนที่ GlobalSurfaceWater ที่ถูกกรองเฉพาะช่วงเวลา ลงในแผนที่
Map.addLayer(gsw_filtered, {'bands': 'occurrence', 'min': 0, 'max': 100, 'palette': ['red', 'blue']}, 'Surface Water Occurrence')

# ข้อมูลคุณภาพอากาศ

# ก๊าซที่มีผลกระทบต่อคุณภาพอากาศ เช่น คาร์บอนมอนอกไซด์ (CO), ไนโตรเจนไดออกไซด์ (NO2), และซัลเฟอร์ไดออกไซด์ (SO2)

โค้ดนี้เป็นการใช้ Google Earth Engine เพื่อดึงข้อมูลด้านสิ่งแวดล้อมเกี่ยวกับคุณภาพอากาศ โดยเน้นไปที่ประเภทของก๊าซที่มีผลกระทบต่อคุณภาพอากาศ เช่น คาร์บอนมอนอกไซด์ (CO), ไนโตรเจนไดออกไซด์ (NO2), และซัลเฟอร์ไดออกไซด์ (SO2) ในพื้นที่ของ 9 จังหวัดภาคเหนือในประเทศไทยในช่วงปี 2019-2023 โดยกำหนดวันที่เริ่มต้นและสิ้นสุด และกรองข้อมูลตามขอบเขตของพื้นที่ที่สนใจ (province_boundary) ซึ่งมีการแสดงผลผลลัพธ์บนแผนที่ผ่าน Geemap library ที่มีการกำหนดตัวค่าพารามิเตอร์การแสดงผลแบบสีของแต่ละประเภทก๊าซ ด้วยการใช้พาเลทสีที่แสดงระดับค่าต่าง ๆ ของก๊าซที่วัดได้ตามที่กำหนดไว้ และมีการเพิ่มตัวควบคุมเลเยอร์ในแผนที่เพื่อให้ผู้ใช้สามารถเปิด-ปิดเลเยอร์ต่าง ๆ ได้ตามความต้องการ

In [31]:
# กำหนดปีเริ่มและปีสิ้นสุด
start_year = 2019
end_year = 2023

# สร้างวัตถุวันที่สองปีสำหรับปีเริ่มและปีสิ้นสุด
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year + 1, 1, 1)

# สร้างรายการปี
years = ee.List.sequence(start_year, end_year)

# สร้างรายการเดือน
months = ee.List.sequence(1, 12)

# สร้างช่วงวันที่
date_range = ee.DateRange(start_date, end_date)

# กำหนด ImageCollections
collection1 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CO")\
.filterDate(date_range)\
.filterBounds(province_boundary)\
.select('CO_column_number_density')\

collection2 = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2")\
.filterDate(date_range)\
.filterBounds(province_boundary)\
.select('tropospheric_NO2_column_number_density')\

collection3 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_SO2")\
.filterDate(date_range)\
.filterBounds(province_boundary)\
.select('SO2_column_number_density')\

# ลด ImageCollections เพื่อให้ได้ภาพที่มีค่าเฉลี่ย
CO_img   = collection1.reduce(ee.Reducer.mean()).clip(province_boundary)
NO2_img = collection2.reduce(ee.Reducer.mean()).clip(province_boundary)
SO2_img =  collection3.reduce(ee.Reducer.mean()).clip(province_boundary)

# กำหนดพารามิเตอร์การแสดงผล
CO_viz = {
'min': 0.015,
'max': 0.033,
'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

NO2_viz = {
'min': 0.000,
'max': 0.001,
'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

SO2_viz = {
'min': -0.00009,
'max': 0.0005,
'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# สร้างแผนที่ด้วย Geemap (ศูนย์กลางที่ [16.828023952686376, 100.41638539390868], ระดับซูมเป็น 8)
Map = geemap.Map(center=[16.828023952686376, 100.41638539390868], zoom=8)

# เพิ่มชั้นข้อมูล คาร์บอนมอนอกไซด์ (CO) ลงในแผนที่
Map.addLayer(CO_img, CO_viz, 'S5P offline Carbon Monoxide(CO)');

# เพิ่มชั้นข้อมูล ไนโตรเจนไดออกไซด์ (NO2) ลงในแผนที่
Map.addLayer(NO2_img, NO2_viz, 'S5P offline Nitrogen Dioxide(NO2)');

# เพิ่มชั้นข้อมูล ซัลเฟอร์ไดออกไซด์ (SO2) ลงในแผนที่
Map.addLayer(SO2_img, SO2_viz, 'S5P offline Sulfur Dioxide(SO2)');

# เพิ่มตัวควบคุมเลเยอร์ในแผนที่
Map.addLayerControl()

# แสดงแผนที่
Map

Map(center=[16.828023952686376, 100.41638539390868], controls=(WidgetControl(options=['position', 'transparent…

# ฝุ่น PM2.5

ในโค้ดนี้ เริ่มต้นด้วยการกำหนดวันที่เริ่มต้นและวันที่สิ้นสุดของช่วงเวลาที่ต้องการดึงข้อมูลจาก Earth Engine ซึ่งในที่นี้คือชุดข้อมูล Terra & Aqua MAIAC Land Aerosol Optical Depth Daily 1 km ของ MODIS โดยทำการกรองตามช่วงเวลาและพื้นที่ที่กำหนด และเลือกดึงเฉพาะแบนด์ 'Optical_Depth_047' ซึ่งเป็นข้อมูล Aerosol Optical Depth (AOD) ที่คำนวณได้จากรางวัลแสงในช่วงคลื่น 0.47 μm ของกล้อง

หลังจากนั้น โค้ดทำการปรับใช้ตัวคูณในการแปลงข้อมูล AOD ให้อยู่ในหน่วยที่ถูกต้อง (0.001) และนำมาเพิ่มเป็นแบนด์ในภาพ ต่อมาได้ทำการนำภาพ AOD มาประมวลผลรวมเข้าด้วยกันในช่วงเวลาที่เลือก และทำการคลิปตามขอบเขตพื้นที่ที่กำหนดไว้ในตัวแปร province_boundary เพื่อให้ได้ภาพที่สะอาดและเหมาะสมกับพื้นที่ที่สนใจ

หลังจากนั้น ทำการนำภาพ AOD ที่ได้มาเพื่อทำการแปลงเป็นข้อมูล PM2.5 โดยใช้สมการความสัมพันธ์ที่กำหนดไว้ และนำมาเพิ่มเป็นแบนด์ในภาพ PM2.5 ทำให้สามารถที่จะแสดงข้อมูล PM2.5 ได้ในแผนที่

ในส่วนของการแสดงผล (visualization) ได้กำหนดค่าต่าง ๆ ในการแสดงผล AOD และ PM2.5 ในแผนที่ เช่น สีและช่วงค่าที่แสดงผล เพื่อให้สามารถระบุระดับของ AOD และ PM2.5 ได้ด้วยสีในแผนที่อย่างชัดเจน และนำมาเพิ่มลงในแผนที่ที่สร้างขึ้นด้วย Geemap

In [32]:
# กำหนดวันที่เริ่มต้นและวันที่สิ้นสุด
dStart = ee.Date('2019-01-01')
dEnd = ee.Date(datetime.now())

# ชุดข้อมูล Terra & Aqua MAIAC Land Aerosol Optical Depth Daily 1 km
dataset = ee.ImageCollection('MODIS/006/MCD19A2_GRANULES') \
    .filterDate(dStart, dEnd) \
    .filter(ee.Filter.bounds(province_boundary)) \
    .select('Optical_Depth_047')

# ปรับใช้ตัวคูณ
def applyScaleFactors(image):
    Depth_047Bands = image.select('Optical_Depth_047').multiply(0.001)
    return image.addBands(Depth_047Bands, None, True)

# ประมวลผลข้อมูลราวเซ็น
collection1 = dataset.map(applyScaleFactors)
AOD_img = collection1.mosaic().clip(province_boundary)

#  แปลง Terra & Aqua MAIAC เป็น PM 2.5
# ใช้สมการความสัมพันธ์: y = 23.89x + 5.18
def transform_to_pm25(image):
    tpm25 = image.select('Optical_Depth_047').multiply(0.001).multiply(23.89).add(5.18)
    return image.addBands(tpm25, None, True)

# ประมวลผลข้อมูลราวเซ็น
collection2 = dataset.map(transform_to_pm25)
PM25_img = collection2.mosaic().clip(province_boundary)

#  กำหนดสี palette
# โดย palette นี้จะถูกนำมาใช้ในการแสดงผลภาพ
AOD_vis = {
    'min': 0.029,
    'max': 1.476,
    'palette': ['white', 'yellow', 'orange', 'red', 'purple', 'black']
}
pm25_vis = {
    'min': 5.873,
    'max': 40.434,
    'palette': ['white', 'yellow', 'orange', 'red', 'purple', 'black']
}

# เพิ่มชั้นข้อมูล  Aerosol Optical Depth (AOD) ลงในแผนที่
Map.addLayer(AOD_img, AOD_vis, ' Aerosol optical depth over land (0.47 μm)');

# เพิ่มชั้นข้อมูล PM2.5 ลงในแผนที่
Map.addLayer(PM25_img, pm25_vis, ' Satellite-Based PM2.5');